In [1]:
import pandas as pd
import os
import datetime
import numpy as np
import gc
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Compare_Q4_Before_After_Groups_JT_20190327'

In [2]:
Q3_End=datetime.date(2018,11,3)
Q4_End=datetime.date(2019,2,2)

lapsed_begin=Q3_End-datetime.timedelta(days=4*365+1)
lapsed_begin

datetime.date(2014, 11, 3)

In [3]:
lapsed=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/lapsed20140826_20170226/MediaStormLapsedCustDtl.txt",
                     sep=",",usecols=['customer_id_hashed','transaction_date'],dtype=str).drop_duplicates()
lapsed=lapsed[lapsed['transaction_date']>=str(lapsed_begin)]

lapsed['transaction_date'].min()

'2014-11-03'

In [4]:
chunksize_num = 10**7
filename='/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update/crm_newscore_0922/combinedtransactions_0922.csv'
dftrans_before_20180922=pd.DataFrame()
count_i=0

for chunk in pd.read_csv(filename, chunksize=chunksize_num,dtype=str):
    chunk['total_transaction_amt']=chunk['total_transaction_amt'].astype(float)
    chunk['total_transaction_units']=chunk['total_transaction_units'].astype(float)
    chunk = chunk[['customer_id_hashed','transaction_date']].drop_duplicates()
    dftrans_before_20180922=dftrans_before_20180922.append(chunk)
    count_i+=1
    print(count_i,datetime.datetime.now())
    

1 2019-03-30 17:02:50.227576
2 2019-03-30 17:03:27.172147
3 2019-03-30 17:04:01.409444
4 2019-03-30 17:04:39.549824
5 2019-03-30 17:05:18.411947
6 2019-03-30 17:05:57.279613
7 2019-03-30 17:06:36.328740
8 2019-03-30 17:07:16.129066
9 2019-03-30 17:07:56.637125
10 2019-03-30 17:08:38.292524
11 2019-03-30 17:09:19.662925
12 2019-03-30 17:10:02.798904
13 2019-03-30 17:10:47.167229
14 2019-03-30 17:11:32.417311
15 2019-03-30 17:12:15.199163


In [5]:
all_rewards_most_recent=dftrans_before_20180922.append(lapsed)
all_rewards_most_recent=all_rewards_most_recent.sort_values(["customer_id_hashed","transaction_date"],ascending=[True,False])
all_rewards_most_recent=all_rewards_most_recent.drop_duplicates(['customer_id_hashed'])
all_rewards_most_recent.shape

(23136029, 2)

In [6]:
def recrusive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)
# Before the end of Q4 only            
historical_daily_data_folder="/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/"
historical_daily_data_list=list(recrusive_file_gen(historical_daily_data_folder))
historical_daily_data_list=[x for x in historical_daily_data_list if (".txt" in x) & ("DailySales" in x)]
historical_daily_df=pd.DataFrame({"file_path":historical_daily_data_list})
historical_daily_df['week_end_dt']=historical_daily_df['file_path'].apply(lambda x: x.split(".")[0].split("MediaStormDailySalesHistory")[1])
historical_daily_df['week_end_dt']=historical_daily_df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y%m%d").date())
historical_daily_df=historical_daily_df[historical_daily_df['week_end_dt']<=datetime.date(2019,2,2)]
historical_daily_df=historical_daily_df[historical_daily_df['week_end_dt']>datetime.date(2018,9,22)]


In [7]:
historical_daily_df_20180929_20181103=historical_daily_df[historical_daily_df['week_end_dt']<=datetime.date(2018,11,3)]
historical_daily_df_20181110_20190202=historical_daily_df[historical_daily_df['week_end_dt']>=datetime.date(2018,11,10)]

In [8]:
rew_df_after_20180929_20181103=pd.DataFrame()
rew_df_after_20181110_20190202=pd.DataFrame()


for file in historical_daily_df_20180929_20181103['file_path'].tolist():
    df=pd.read_table(file,sep="|",dtype=str,usecols=["customer_id_hashed","transaction_dt"]).drop_duplicates()
    df=df[~pd.isnull(df['customer_id_hashed'])]
    rew_df_after_20180929_20181103=rew_df_after_20180929_20181103.append(df)
    print(file,datetime.datetime.now())
rew_df_after_20180929_20181103=rew_df_after_20180929_20181103.rename(columns={"transaction_dt":"transaction_date"})

for file in historical_daily_df_20181110_20190202['file_path'].tolist():
    df=pd.read_table(file,sep="|",dtype=str,usecols=["customer_id_hashed","transaction_dt"]).drop_duplicates()
    df=df[~pd.isnull(df['customer_id_hashed'])]
    rew_df_after_20181110_20190202=rew_df_after_20181110_20190202.append(df)
    print(file,datetime.datetime.now())
rew_df_after_20181110_20190202=rew_df_after_20181110_20190202.rename(columns={"transaction_dt":"transaction_date"})

/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180929.txt 2019-03-30 17:28:25.119492
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181006.txt 2019-03-30 17:28:53.535690
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181013.txt 2019-03-30 17:29:14.277381
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181020.txt 2019-03-30 17:29:35.427964
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181027.txt 2019-03-30 17:29:58.537136
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181103.txt 2019-03-30 17:30:22.278057
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181110.txt 2019-03-30 17:30:49.272289
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181117.txt 2019-03-30 17:31:18.836521


In [9]:
all_rewards_most_recent=all_rewards_most_recent.append(rew_df_after_20180929_20181103)
all_rewards_most_recent=all_rewards_most_recent.sort_values(["customer_id_hashed","transaction_date"],ascending=[True,False])
all_rewards_most_recent=all_rewards_most_recent.drop_duplicates(['customer_id_hashed'])
all_rewards_most_recent['transaction_date_before_Q4']=all_rewards_most_recent['transaction_date']

all_rewards_most_recent_After=all_rewards_most_recent[['customer_id_hashed','transaction_date']].append(rew_df_after_20181110_20190202)
all_rewards_most_recent_After=all_rewards_most_recent_After.sort_values(["customer_id_hashed","transaction_date"],ascending=[True,False])
all_rewards_most_recent_After=all_rewards_most_recent_After.drop_duplicates(['customer_id_hashed']).rename(columns={"transaction_date":"transaction_date_after_Q4"})

all_rewards_most_recent=pd.merge(all_rewards_most_recent,all_rewards_most_recent_After,on="customer_id_hashed",how="outer")
del all_rewards_most_recent_After

gc.collect()

79

In [10]:
all_rewards_most_recent.head(2)

,customer_id_hashed,transaction_date,transaction_date_before_Q4,transaction_date_after_Q4
0,00000135f48c68690ad3d5fc9ada41bb5cd687452007e8...,2018-10-30,2018-10-30,2018-10-30
1,000001dadc0265bf9d250566d74e0006323f18b5826641...,2018-10-27,2018-10-27,2018-12-21


In [11]:
all_rewards_most_recent[pd.isnull(all_rewards_most_recent['transaction_date_before_Q4'])].shape

(1853494, 4)

In [12]:
all_rewards_most_recent[pd.isnull(all_rewards_most_recent['transaction_date_after_Q4'])].shape

(0, 4)

In [13]:
(all_rewards_most_recent['transaction_date']==all_rewards_most_recent['transaction_date_after_Q4']).unique()

array([ True, False], dtype=bool)

In [14]:
all_rewards_most_recent.shape

(25642058, 4)

In [15]:
all_rewards_most_recent['transaction_date_before_Q4']=all_rewards_most_recent['transaction_date_before_Q4'].fillna("2019-12-31")
all_rewards_most_recent['transaction_date_before_Q4']=pd.to_datetime(all_rewards_most_recent['transaction_date_before_Q4'],format='%Y-%m-%d')
all_rewards_most_recent['Days_to_Q3_End']=Q3_End-all_rewards_most_recent['transaction_date_before_Q4']
# Filled the na before with a later adte 2019-12-31


all_rewards_most_recent['transaction_date_after_Q4']=pd.to_datetime(all_rewards_most_recent['transaction_date_after_Q4'],format='%Y-%m-%d')
all_rewards_most_recent['Days_to_Q4_End']=Q4_End-all_rewards_most_recent['transaction_date_after_Q4']
print(datetime.datetime.now())

2019-03-30 17:58:06.353142


In [16]:
all_rewards_most_recent['Month_to_Q3_End']=all_rewards_most_recent['Days_to_Q3_End'].apply(lambda x: x.days/(365.25/12))
all_rewards_most_recent['Month_to_Q4_End']=all_rewards_most_recent['Days_to_Q4_End'].apply(lambda x: x.days/(365.25/12))
print(datetime.datetime.now())

2019-03-30 18:06:54.120088


In [17]:
sorted(all_rewards_most_recent['Month_to_Q3_End'].unique().tolist())[:3]

[-13.897330595482547, 0.0, 0.03285420944558522]

In [18]:
all_rewards_most_recent['Group_before_Q4']=np.where((all_rewards_most_recent['Month_to_Q3_End']>12) & (all_rewards_most_recent['Month_to_Q3_End']<=48),"Lapsed_12_48",
                                                   np.where(all_rewards_most_recent['Month_to_Q3_End']>48,"WD_48+",
                                                            np.where((all_rewards_most_recent['Month_to_Q3_End']>=0) & (all_rewards_most_recent['Month_to_Q3_End']<=12),"Active",
                                                                    np.where(all_rewards_most_recent['Month_to_Q3_End']<0,"NotAvailable_Before_Q4","NaN")
                                                                    )
                                                           )
                                                   )
all_rewards_most_recent['Group_after_Q4']=np.where((all_rewards_most_recent['Month_to_Q4_End']>12) & (all_rewards_most_recent['Month_to_Q4_End']<=48),"Lapsed_12_48",
                                                   np.where(all_rewards_most_recent['Month_to_Q4_End']>48,"WD_48+",
                                                            np.where((all_rewards_most_recent['Month_to_Q4_End']>=0) & (all_rewards_most_recent['Month_to_Q4_End']<=12),"Active",
                                                                    np.where(all_rewards_most_recent['Month_to_Q4_End']<0,"NotAvailable_Before_Q4","NaN")
                                                                    )
                                                           )
                                                   )

In [19]:
all_rewards_most_recent['Group_before_Q4'].unique()

array(['Active', 'Lapsed_12_48', 'NotAvailable_Before_Q4'], dtype=object)

In [20]:
all_rewards_most_recent['Group_after_Q4'].unique()

array(['Active', 'WD_48+', 'Lapsed_12_48'], dtype=object)

In [21]:
Q4_sales_by_id=pd.DataFrame()
Q4_files_df=historical_daily_df[historical_daily_df['week_end_dt']>=datetime.date(2018,11,10)]
print(Q4_files_df.shape)
for file in Q4_files_df['file_path'].tolist():
    df=pd.read_table(file,sep="|",usecols=['customer_id_hashed','item_transaction_amt'])
    df=df[~pd.isnull(df['customer_id_hashed'])]
    df=df.groupby(['customer_id_hashed'])['item_transaction_amt'].sum().to_frame().reset_index()
    
    Q4_sales_by_id=Q4_sales_by_id.append(df)
    print(file,datetime.datetime.now())
Q4_sales_by_id=Q4_sales_by_id.groupby("customer_id_hashed")['item_transaction_amt'].sum().to_frame().reset_index()
Q4_sales_by_id=Q4_sales_by_id.rename(columns={"item_transaction_amt":"sales_2018_Q4"})
Q4_sales_by_id['Q4_Shopping_Group']="Q4_Shopped"

(13, 2)
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181110.txt 2019-03-30 18:07:49.841098
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181117.txt 2019-03-30 18:08:25.668803
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181124.txt 2019-03-30 18:09:11.270595
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181201.txt 2019-03-30 18:09:57.134007
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181208.txt 2019-03-30 18:10:47.659399
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181215.txt 2019-03-30 18:11:38.230464
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181222.txt 2019-03-30 18:12:39.303698
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181229.txt 2019-03-30 18:13:19

In [22]:
all_rewards_most_recent=pd.merge(all_rewards_most_recent,Q4_sales_by_id,on="customer_id_hashed",how="outer")
all_rewards_most_recent.shape

(25642058, 12)

In [23]:
all_rewards_most_recent.head(2)

,customer_id_hashed,transaction_date,transaction_date_before_Q4,transaction_date_after_Q4,Days_to_Q3_End,Days_to_Q4_End,Month_to_Q3_End,Month_to_Q4_End,Group_before_Q4,Group_after_Q4,sales_2018_Q4,Q4_Shopping_Group
0,00000135f48c68690ad3d5fc9ada41bb5cd687452007e8...,2018-10-30,2018-10-30,2018-10-30,4 days,95 days,0.131417,3.121150,Active,Active,NaN,NaN
1,000001dadc0265bf9d250566d74e0006323f18b5826641...,2018-10-27,2018-10-27,2018-12-21,7 days,43 days,0.229979,1.412731,Active,Active,42.2,Q4_Shopped


In [24]:
Q4_new_sign_ups_files=list(recrusive_file_gen("/home/jian/BigLots/2018_by_weeks/"))+list(recrusive_file_gen("/home/jian/BigLots/2019_by_weeks/"))

Q4_new_sign_ups_files=[x for x in Q4_new_sign_ups_files if "Master" in x]
Q4_new_sign_ups_files_df=pd.DataFrame({"file_path":Q4_new_sign_ups_files})
Q4_new_sign_ups_files_df['week_end_dt']=Q4_new_sign_ups_files_df['file_path'].apply(lambda x: x.split("_by_weeks/MediaStorm_")[1][:10])
Q4_new_sign_ups_files_df=Q4_new_sign_ups_files_df[Q4_new_sign_ups_files_df['week_end_dt']<="2019-02-02"]
Q4_new_sign_ups_files_df=Q4_new_sign_ups_files_df[Q4_new_sign_ups_files_df['week_end_dt']>="2018-11-04"]

all_new_sign_ups=pd.DataFrame()
for file in Q4_new_sign_ups_files_df['file_path'].unique().tolist():
    df=pd.read_table(file,sep="|",usecols=['customer_id_hashed','sign_up_date'])
    all_new_sign_ups=all_new_sign_ups.append(df)
all_new_sign_ups.shape

all_new_sign_ups['NewRewards']="Q4_New_Sign_Ups"
all_new_sign_ups=all_new_sign_ups.drop_duplicates("customer_id_hashed")

In [25]:
all_rewards_most_recent=pd.merge(all_rewards_most_recent,all_new_sign_ups,on="customer_id_hashed",how="outer")
all_rewards_most_recent['NewRewards']=all_rewards_most_recent['NewRewards'].fillna("Old_Rewards_Members")
all_rewards_most_recent['sales_2018_Q4']=all_rewards_most_recent['sales_2018_Q4'].fillna(0)
all_rewards_most_recent['Q4_Shopping_Group']=all_rewards_most_recent['Q4_Shopping_Group'].fillna("Q4_No_Shop")

all_rewards_most_recent.shape

(25655008, 14)

In [26]:
output_id_count=all_rewards_most_recent.groupby(['Group_before_Q4','Group_after_Q4','NewRewards','Q4_Shopping_Group'])['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"id_count"})
output_id_Q4_sales=all_rewards_most_recent.groupby(['Group_before_Q4','Group_after_Q4','NewRewards','Q4_Shopping_Group'])['sales_2018_Q4'].sum().to_frame().reset_index()

output=pd.merge(output_id_count,output_id_Q4_sales,on=['Group_before_Q4','Group_after_Q4','NewRewards','Q4_Shopping_Group'],how="outer")
output


,Group_before_Q4,Group_after_Q4,NewRewards,Q4_Shopping_Group,id_count,sales_2018_Q4
0,Active,Active,Old_Rewards_Members,Q4_No_Shop,7418155,0.000000e+00
1,Active,Active,Old_Rewards_Members,Q4_Shopped,7188995,6.955726e+08
2,Active,Active,Q4_New_Sign_Ups,Q4_No_Shop,3,0.000000e+00
3,Active,Active,Q4_New_Sign_Ups,Q4_Shopped,267,4.954848e+04
4,Active,Lapsed_12_48,Old_Rewards_Members,Q4_No_Shop,1864701,0.000000e+00
5,Lapsed_12_48,Active,Old_Rewards_Members,Q4_Shopped,735197,5.545576e+07
6,Lapsed_12_48,Lapsed_12_48,Old_Rewards_Members,Q4_No_Shop,6487113,0.000000e+00
7,Lapsed_12_48,WD_48+,Old_Rewards_Members,Q4_No_Shop,94133,0.000000e+00
8,NotAvailable_Before_Q4,Active,Old_Rewards_Members,Q4_Shopped,570221,5.002389e+07
9,NotAvailable_Before_Q4,Active,Q4_New_Sign_Ups,Q4_Shopped,1283273,1.378559e+08


In [27]:
output.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Compare_Q4_Before_After_Groups_JT_20190327/BL_Q4_rewards_migration_output_summary_JL_"+str(datetime.datetime.now().date())+".csv",index=False)



In [28]:
all_rewards_most_recent.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Compare_Q4_Before_After_Groups_JT_20190327/BL_Q4_rewards_migration_rewards_data_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


In [29]:
all_rewards_most_recent.shape

(25655008, 14)

In [30]:
len(all_rewards_most_recent['customer_id_hashed'].unique())

25655008